In [1]:
import pandas as pd
from statistics import mean, median
import numpy as np


In [8]:
ty = "proposition"
nums = [2, 4, 8, 16, 32]
seeds = [42, 52, 78, 121]


for num in nums:
    FPs = []
    FNs = []
    perc_FP = []
    perc_FN = []
    for seed in seeds:
        for run in range(1,4):
            pred_df = pd.read_csv(f"./predictions/Llama3-8B_{ty}_{num}_{seed}_{run}.tsv", sep="\t").dropna()

            preds = [pred.lower().replace("**","").replace(".","").replace("_","").replace("'","").strip() for pred in list(pred_df["prediction"])]

            #print(set(preds))
            
            for i in range(len(preds)):
                pred = preds[i]
                if "self-contr" in pred:
                    preds[i] = "self-contradiction"
                elif "non-contr" in pred:
                    preds[i] = "non-contradiction"

            #print(set(preds))

            loc_status = []

            pred_df["prediction"] = preds

            for gold, pred in zip(pred_df["gold_label"], preds):
                #print(pred, type(pred))
                if gold == "self-contradiction" and pred == "self-contradiction":
                    loc_status.append("TP")
                elif gold == "non-contradiction" and pred == "non-contradiction":
                    loc_status.append("TN")
                elif gold == "self-contradiction" and pred == "non-contradiction":
                    loc_status.append("FN")
                elif gold == "non-contradiction" and pred == "self-contradiction":
                    loc_status.append("FP")

            pred_df["eval_status"] = loc_status

            preds_FP = pred_df[pred_df["eval_status"] == "FP"]
            preds_FN = pred_df[pred_df["eval_status"] == "FN"]

            preds_FN.to_csv(f"erroranalysis/Llama3-8B_FN_{ty}_{num}_{seed}_{run}.tsv", sep="\t", index=False)
            preds_FP.to_csv(f"erroranalysis/Llama3-8B_FP_{ty}_{num}_{seed}_{run}.tsv", sep="\t", index=False)

            FP = pred_df["eval_status"].value_counts()["FP"]
            FN = pred_df["eval_status"].value_counts()["FN"]
            count_all = len(list(pred_df["gold_label"]))

            FPs.append(FP)
            perc_FP.append(FP/count_all)

            FNs.append(FN)
            perc_FN.append(FN/count_all)
            
    print(num, "-Shot ", ty)
    #print(FPs)
    print(f"FP: {round(100*mean(perc_FP),2)}\\pm{round(100*np.std(np.asarray(perc_FP)),2)} ({mean(FPs)}\\pm {round(np.std(np.asarray(FPs)),2)})")
    #print(f"FP: {median(FPs)}\\pm\{round(np.std(np.asarray(FPs)),2)} ({100*median(perc_FP)}\\pm{100*np.std(np.asarray(perc_FP))})")

    #print(FNs)
    print(f"FN: {round(100*mean(perc_FN),2)}\\pm{round(100*np.std(np.asarray(perc_FN)),2)} ({mean(FNs)}\\pm\{round(np.std(np.asarray(FNs)),2)})")
    #print(f"FN: {median(FNs)}\\pm\{round(np.std(np.asarray(FNs)),2)} ({100*median(perc_FN)}\\pm{100*np.std(np.asarray(perc_FN))})")
    

2 -Shot  proposition
FP: 28.99\pm5.23 (383\pm 69.19)
FN: 15.38\pm5.22 (203\pm\69.05)
4 -Shot  proposition
FP: 27.2\pm5.81 (359\pm 76.81)
FN: 16.69\pm4.73 (220\pm\62.52)
8 -Shot  proposition
FP: 26.3\pm9.69 (346\pm 127.6)
FN: 17.07\pm8.37 (224\pm\110.18)
16 -Shot  proposition
FP: 33.68\pm7.33 (440\pm 96.0)
FN: 10.75\pm5.81 (140\pm\76.06)
32 -Shot  proposition
FP: 27.82\pm9.63 (359\pm 124.45)
FN: 16.23\pm9.08 (209\pm\117.38)


In [2]:
types = ["locution", "proposition"]

for ty in types:
    FN_best = pd.read_csv(f"erroranalysis/gpt3_FN_{ty}_2_42_1.tsv", sep="\t")
    FP_best = pd.read_csv(f"erroranalysis/Llama3-8B_FP_{ty}_2_42_1.tsv", sep="\t")

    FN_best_100 = FN_best.sample(random_state=42, frac=0.5).to_csv(f"gpt3_FN_{ty}_2_42_1_100.tsv", index=False, sep="\t")
    FP_best_100 = FP_best.sample(random_state=42, frac=0.5).to_csv(f"gpt3_FP_{ty}_2_42_1_100.tsv", index=False, sep="\t")


FileNotFoundError: [Errno 2] No such file or directory: 'erroranalysis/gpt3_FN_locution_2_42_1.tsv'

In [21]:
ty = "proposition"

FPs = []
FNs = []
perc_FP = []
perc_FN = []
    
for run in range(1,4):
    pred_df = pd.read_csv(f"../zeroShot_predictions/Llama3-8B_zeroShot_{ty}_preds_{run}.tsv", sep="\t").dropna()

    preds = [pred.lower().replace("**","").replace(".","").replace("_","").replace("'","").strip() for pred in list(pred_df["pred"])]

    print(set(preds))

    for i in range(len(preds)):
        pred = preds[i]
        if "self-contr" in pred:
            preds[i] = "self-contradiction"
        elif "non-contr" in pred:
            preds[i] = "non-contradiction"

    print(set(preds))

    loc_status = []

    pred_df["prediction"] = preds

    for gold, pred in zip(pred_df["gold_label"], preds):
        #print(pred, type(pred))
        if gold == 1 and pred == "self-contradiction":
            loc_status.append("TP")
        elif gold == 0 and pred == "non-contradiction":
            loc_status.append("TN")
        elif gold == 1 and pred == "non-contradiction":
            loc_status.append("FN")
        elif gold == 0 and pred == "self-contradiction":
            loc_status.append("FP")

    pred_df["eval_status"] = loc_status

    preds_FP = pred_df[pred_df["eval_status"] == "FP"]
    preds_FN = pred_df[pred_df["eval_status"] == "FN"]

    preds_FN.to_csv(f"erroranalysis/Llama3-8B_FN_zeroShot_{run}.tsv", sep="\t", index=False)
    preds_FP.to_csv(f"erroranalysis/Llama3-8B_FP_zeroShot_{run}.tsv", sep="\t", index=False)

    FP = pred_df["eval_status"].value_counts()["FP"]
    FN = pred_df["eval_status"].value_counts()["FN"]
    count_all = len(list(pred_df["gold_label"]))

    FPs.append(FP)
    perc_FP.append(FP/count_all)

    FNs.append(FN)
    perc_FN.append(FN/count_all)

print("zero-Shot ", ty)
#print(FPs)
print(f"FP: {round(100*mean(perc_FP),2)}\\pm{round(100*np.std(np.asarray(perc_FP)),2)} ({mean(FPs)}\\pm {round(np.std(np.asarray(FPs)),2)})")
#print(f"FP: {median(FPs)}\\pm\{round(np.std(np.asarray(FPs)),2)} ({100*median(perc_FP)}\\pm{100*np.std(np.asarray(perc_FP))})")

#print(FNs)
print(f"FN: {round(100*mean(perc_FN),2)}\\pm{round(100*np.std(np.asarray(perc_FN)),2)} ({mean(FNs)}\\pm{round(np.std(np.asarray(FNs)),2)})")
#print(f"FN: {median(FNs)}\\pm\{round(np.std(np.asarray(FNs)),2)} ({100*median(perc_FN)}\\pm{100*np.std(np.asarray(perc_FN))})")
    

{'self-contradiction', 'non-contradiction'}
{'self-contradiction', 'non-contradiction'}
{'self-contradiction', 'non-contradiction'}
{'self-contradiction', 'non-contradiction'}
{'self-contradiction', 'non-contradiction'}
{'self-contradiction', 'non-contradiction'}
zero-Shot  proposition
FP: 5.01\pm0.22 (66\pm 2.87)
FN: 39.3\pm0.09 (520\pm1.25)
